In [2]:
##IMPORT REQUIRED MODULES 

from bs4 import BeautifulSoup
import requests, ssl
import openpyxl
import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication


In [3]:
##GENERATE AN EXCEL FILE TP SAVE OUR RESULTS

today = str(datetime.date.today())
excel = openpyxl.Workbook()
sheet = excel.active 
sheet.title='jobs_on_'+ today

In [4]:
##FUNCTION getlatestjobs SCRAPS THE INDEED WEBSITE FOR LATEST JOBS, BASED ON JOBTITLE AND PLACE

def getlatestjobs(job,place):

            url =f'https://in.indeed.com/jobs?q={job}&l={place}&sort=date'   

            r = requests.get(url)

            soup = BeautifulSoup(r.content, 'html.parser')

            try: 
                lists = soup.find('div', id='mosaic-provider-jobcards').find_all('div', class_='job_seen_beacon')
            except:
                return(print("no jobsfound"))
            
            for a in lists: 
                if  not a.find('td', class_ ='resultContent').find('h2',class_='jobTitle').find('div',class_='new'):
                    jobname = a.find('td', class_ ='resultContent').find('h2',class_='jobTitle').text
                    
                else: 
                    jobname = a.find('td', class_ ='resultContent').find('h2',class_='jobTitle').find_all('span')[1].text
                companyname = a.find('span', class_= "companyName").text.replace(",","..")
                joblocation = a.find('div', class_="companyLocation").get_text(strip=True)
                dateposted =  a.find('span', class_="date").text.replace("Posted","")
                try:
                    salary = a.find('div',class_= "metadata").get_text(strip=True)
                except:
                    salary = "Not Specified" 
                sheet.append([jobname,companyname,joblocation,dateposted,salary])
                ####save the scrapped data to an excel file 
                excel.save("latestjobs.xlsx")

    



In [5]:
###Module to send the extracted results to gmail account

def sendmail():
   ####to get latest jobs to gmail place your gmail account password instead of xxxxxxx 
    email_from = 'kalyanpro198@gmail.com'
    password = 'prasana16'
    email_to = 'kalyanpro198@gmail.com'

    def attach_file_to_email(email_message, filename):
    
        with open(filename, "rb") as f:
            file_attachment = MIMEApplication(f.read())
        # Attach the file to the message
        email_message.attach(file_attachment)

    ##set up the From, To, Subject fields
    email_message = MIMEMultipart()
    email_message['From'] = email_from
    email_message['To'] = email_to
    email_message['Subject'] = "Latest data analyst jobs -" + today

    attach_file_to_email(email_message, 'latestjobs.xlsx')

    email_string = email_message.as_string()

    # Connect to the Gmail smtp server and Send Email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(email_from, password)
        server.sendmail(email_from, email_to, email_string)

sendmail()

In [ ]:
import time
####To get the latest Jobs thorugh mail every day, here job = "data analyst" & "chennai"
while True:
    getlatestjobs('data analyst','chennai')
    sendmail()
    time.sleep(60*60*24)